## How to scrape confluence


In [ ]:
!uv pip install atlassian-python-api

In [ ]:
from atlassian import Confluence
import json
import sys, time
import re

confluence_umbrella = Confluence(
    url='https://confluence-eng-rtp2.cisco.com/conf/',
    token="____",
    backoff_and_retry=True
    )

confluence_rtp = Confluence(
    url='https://confluence.it.umbrella.com/',
    token="____",
    backoff_and_retry=True
    )

confluence = confluence_rtp

# To scrape an entire subtree in confluence, you need to get the page ID of the
# top level page. You can get Page ID of any page from the URL that shows up 
# after clicking on the "page information" from the 3 dots menu on the upper 
# right corner
res = confluence.cql('ancestor=31392960', limit=1000)

def save_file(outfile, content):
    """
    # A function to create pdf from byte-stream response
    :param outfile:
    :param content:
    :return:
    """
    file_pdf = open(outfile, 'wb')
    file_pdf.write(content)
    file_pdf.close()
    print("Completed")

def replace_special_chars(text):
    """
    Replaces special characters with underscores using regex.
    """
    return re.sub(r'[^a-zA-Z0-9_.]', '_', text)  # Keep only alphanumeric and underscore

if __name__ == '__main__':
    
    results = res.get('results')
    
    for page in results:
        
        #Get your confluence page as byte-stream
        page_id = page['content']['id']
        page_title = page['content']['title']

        # remove/replace spl characters in the filename
        file_name = replace_special_chars(f"umbrella_{page['content']['title']}_{page['content']['id']}.pdf")
        
        # Call function that will create pdf
        response = confluence.get_page_as_pdf(page['content']['id'])
        
        print(f"writing file {file_name}")
        
        # save file using byte-stream response you received above.
        save_file(file_name, content=response)
        
        # avoid throttleing if any.
        time.sleep(5)
